## Discover IPv6 and 6LoWPAN

In this excercice you will learn how you should communicate with IPV6 and wireless low power devices. The 6LoWPAN protocol has been developed to define the IPv6 adaptation and the way the IP datagrams will be transported over the IEEE802.15.4 radio links. You will deploy a private IPv6 network and test the connectivity between nodes. Moreover you will setup a public IPv6 network with a border router and verify that you can communicate with public servers. Finally with the monitoring feature you will configure a radio sniffer and analyse the traffic. With wireshark, a network protocol analyzer, you will inspect the packets from the different protocols layers with headers and payloads.

### IPv6 overview

Before going into the explanations of network configuration on the nodes, it is essential to have some notions about IPv6. Unlike an IPv4 address which is coded on 32 bits (ie 4 bytes) and uses a decimal notation (for example: 192.168.6.19), an IPv6 address is represented by a series of 128 bits (16 bytes), and is represented with a hexadecimal notation. For example, a public IPv6 address (so-called "global" unicast address, that is to say routable) on IoT-LAB Grenoble site can have the full representation:

`2001:0660:5307:30ff:0000:0000:0000:0001` can be shortened to `2001:660:5307:30ff::1` (a series of 0 contiguous is replaced only once by ::)

This 128-bit series is often divided into 2 parts:

* the least significant 64 bits correspond to the address of the host, `::1` in the previous example. Generally, they are constructed from the MAC address of the host to guarantee the uniqueness of an IPv6 address in a subnet (since the physical address isrmally);

* the most significant 64 bits correspond to the network address, `2001:660:5307:30ff::/64` in the previous example. They contain in particular the "prefix" used for routing IPv6 packets and use, as in IPv4, the CIDR notation: <prefix> / <bit length>. This 64-bit block is divided into a first part containing up to 48 bits and designating the "site prefix",  (`2001:660:5307`) the rest of the bits identifying the subnet (`30ff`). A prefix always contains 64 bits.

Some prefixes are reserved for very specific uses:

* `fe80::/10` is used for unicast addresses called "link-local". This type of address only allows 2 network nodes to communicate if they share a direct physical link. This was the case with two nodes with wireless interface (802.15.4),

* `fd00::/8` is intended for "Unique Local Address" addresses. These addresses correspond to private addresses in IPv4 and are not routable.


### Compile the Contiki-NG firmwares

#### IoT-LAB platform support

Since IoT-LAB boards support is not included in the Contiki-NG official repository, set a ARCH_PATH environment variable that points to the additionnal platform support:

In [ ]:
%env ARCH_PATH = /home/user/iot-lab-training/contiki-ng/iot-lab-contiki-ng/arch

#### Radio settings
If you are running this training as the same time as other people, it is a good idea to change the default radio configuration to avoid too much collision with others.

Use the following cell to give you random values for channel and PAN ID.

In [ ]:
import os,binascii,random
pan_id = binascii.b2a_hex(os.urandom(2)).decode()
channel = random.randint(11, 26)
print('Use CHANNEL={}, PAN_ID=0x{}'.format(channel, pan_id))

Change default values below before running the cell.


In [ ]:
%env RADIO=IEEE802154_CONF_DEFAULT_CHANNEL=26,IEEE802154_CONF_PANID=0xABCD

#### Hello-world

This example has IPv6 stack enabled by default.We modified a bit the hello-world example to enable the shell and use the ping command to test the connectivity between two nodes. . Compile the local example. 

In [ ]:
!make -C hello-world TARGET=iotlab BOARD=m3 DEFINES=$RADIO

#### Border Router

In order for a node to communicate in IPv6 with a host on the global Internet, it needs a “global” unicast address. To do this, a Border Router node must be added to the network, which will be responsible for propagating the IPv6 global prefix to the other nodes. We speak here of border router, because it is on the border between an 802.15.4 network and a classic Ethernet network.

In [ ]:
%env APP_DIR = ../../iot-lab-contiki-ng/contiki-ng/examples/rpl-border-router
!make -C $APP_DIR TARGET=iotlab BOARD=m3 DEFINES=$RADIO

### Launch an experiment

1. Choose your site (grenoble|lille|strasbourg):

In [ ]:
%env SITE=grenoble

2. Submit an experiment with two nodes

In [ ]:
!iotlab-experiment submit -d 120 -l 2,archi=m3:at86rf231+site=$SITE

3. Wait for the experiment to be in the Running state:

In [ ]:
!iotlab-experiment wait

4. Check the nodes allocated to the experiment

In [ ]:
!iotlab-experiment get -ni

### IPv6 connectivity


You can flash the ``Hello World`` firmware on the two nodes with the following command

In [ ]:
!iotlab-node --update hello-world/hello-world.iotlab

At this stage open a Jupyter terminal (use `File > New > Terminal`) two times, one for each node, and connect to the IoT-LAB SSH frontend to have access of the node's serial link. 
On the first shell choose one node and connect to the serial port with ``nc`` command (port 20000). Replace ``<site> <id1> with the good values. Next print the shell usage with the ``help`` command and use ``ip-addr`` command to show IPV6 ``link-local`` address.

On the other node try to ping the IPv6 address and test the connectivity. You can note that we used the IPv6 "link-local" address of the node. We don't have a global IPV6 address and we can only communicate because the two nodes are in the same radio neighborhood with the same physical link (802.15.4 with the same radio channel)

### Communication via a Border Router (BR)

Choose one node of your experiment which will play the role of BR. Replace the ID variable with the good value and flash the firmware only on this node as below


In [ ]:
%env ID=1
!iotlab-node --update $APP_DIR/border-router.iotlab -l $SITE,m3,$ID

use `File > New > Terminal` and connect to the IoT-LAB SSH frontend to have access of the BR node's serial link.

We need to create a network interface and choose a public IPv6 prefix available on the SSH frontend of the site. You can find below the list of available prefix by sites

| Site       | First Prefix       | Last Prefix        | Number of Prefix |
|------------|--------------------|--------------------|--------------------|
| Grenoble   | 2001:660:5307:3100 | 2001:660:5307:317f | 128                |
| Lille      | 2001:660:4403:0480 | 2001:660:4403:04ff | 128                |
| Strasbourg | 2001:660:4701:f0a0 | 2001:660:4701:f0bf | 32                 |

As SSH frontend is a shared environment you must check before. Visualize which prefix are already used

Launch the tunslip6 command with the good IPv6 prefix and node's id for the border router you have chosen. In the example below we specify the m3-10 node and the first prefix of Grenoble site 

This runs the program tunslip6, which bridges the Contiki-NG border router node to the SSH frontend server via a tun interface. Tunslip uses SLIP (Serial Line Internet Protocol) to encapsulate and pass IP traffic to and from the other side of the serial line. The RPL border router, running on the node, has an auto-configured address which you can find out from the above logs. Your host is assigned address ``2001:660:5307:3100::b080``.

To access nodes inside the RPL network, we first need to find out their global IPv6 address. To help with this, the RPL border router runs an HTTP server. You can request it from the SSH frontend:

The page contains a list of routes with the addresses of all nodes in the network.

You can also check that on the other node with the Contiki-NG shell and `ip-addr` command you visualize the global ipv6 address in addition to the ``link-local`` address

The BR node starts advertising the network via RPL DIO (DODAG Information Object) messages. Nodes that receive a DIO message will join the RPL network. Nodes can also search for available networks by sending periodic DIS (DODAG Information Solicitation) messages. You can find more informations about RPL implementation in the [Contiki-NG documentation](https://github.com/contiki-ng/contiki-ng/wiki/Documentation:-RPL)

The last step is to test the IPv6 connectivity with global internet and the node. Try to ping a public Google server

### Radio sniffer

One of the key feature of IoT-LAB is providing automatic monitoring on energy consumption and radio activity, thanks to a Control Node hardware, associated to the experiment node. You do not have to bring a firmware for this Control Node, but just specify its configuration through what we called a 'profile'. Each user can manage his collection of profiles. Here you will create one.

Create a sniffer profile with the good radio channel. Replace CHANNEL variable value with the value obtained at the start.

In [ ]:
%env CHANNEL=11
!iotlab-profile addm3 -n sniff_$CHANNEL -sniffer -channels $CHANNEL

After you have just to update the monitoring configuration of the node which is not the BR. Replace ID variable with the good value

In [ ]:
%env ID=1
!iotlab-node --update-profile sniff_$CHANNEL -l $SITE,m3,$ID

use `File > New > Terminal` and connect to the SSH frontend. Replace `<site>` with the good value.

Launch the sniffer_aggregator command. This command aggregates all the nodes sniffer links (TCP socket on port 30000). By default t it encapsulates packet as ZigBee Encapsulation Protocol (ZEP). With the -r option 802.15.4 payload are extracted and saved in a 802.15.4_link_layer pcap file directly usable in wireshark network traffic analyser. 

Go to the shell node and ping6 Google server

On the sniffer aggregator shell you must see that you have captured packets. Stop it with Ctr^C shortcut when you have about ten packets captured. Edit the pcap file and analyse the traffic.


From the frames you can see the IEEE 802.15.4 data, 6LoWPAN and Internet Protocol version 6 layer.

* In the 802.15.4 section you can see the mac address of the source and destination packet. You can retrieve the mac address of your node which executes the ping6 address in the Src field.

* In the 6LoWPAN section you can see the IPV6 address of the source and destination packet.

* In the IPv6 section you can see the ICMP message which corresponds to a Echo ping request (ping6 command)

From the other frames you should see the ICMP message from the Google server which corresponds to Echo ping reply and you can discover the DODAG Information Solicitation traffic between the border router and the node.
